# Intoduction à Awk

---
<div class="alert alert-block alert-info">
Vous pouvez exécuter les commandes ci-desous directement dans le notebook ou dans un terminal.<br />
Nous vous conseillons de tester les deux.
</div>

**Rappel**, dans un notebook :

- la combinaison de touches <kbd>Ctrl</kbd>+<kbd>Entrée</kbd> exécute une cellule.
- la combinaison de touches <kbd>Shift</kbd>+<kbd>Entrée</kbd> exécute une cellule puis passe à la suivante. C'est équivalent à cliquer sur l'icone ▶️ dans la barre de menu du notebook.
- la combinaison de touches <kbd>Alt</kbd>+<kbd>Entrée</kbd> exécute une cellule puis en crée une nouvelle (vide) en dessous.

Pour ajouter une cellule, vous pouvez aussi cliquer sur l'icone ➕ dans la barre de menu du notebook.

---

`awk` est une outil Unix qui permet de traiter les lignes d'un fichier texte, lu en colonnes. Cela signifie que `awk` lit un fichier ligne par ligne mais peut filtrer et sélectionner des éléments sur la base de colonnes.

La syntaxe de `awk` est :

**`awk [options] 'filtres {actions}' fichier`**

- ⚠️ Les guillemets simples `'` sont très importants.
- Les filtres sont optionnels. Par défaut, tout est sélectionné.
- Les actions sont optionnelles. Par défaut, toute la ligne est affichée.


## Jeu de données

Nous allons utiliser le jeu de données `people.dat` qui contient le sexe, le prénom, la taille et l'âge de différents individus.

Mais au préalable, nous allons nettoyer l'espace de travail. Exécutez la cellule suivante pour supprimer d'éventuels fichiers résiduels :

In [ ]:
rm -f people.* mean_age_women.*

Téléchargez le jeu de données :

In [ ]:
wget https://raw.githubusercontent.com/pierrepo/unix/master/people.dat

Affichez ensuite le contenu du fichier téléchargé :

In [ ]:
cat people.dat

Ce jeu de données contient les caractéristiques d'un certain nombre d'individus :

- La première colonne contient le sexe de la personne (`man` ou `woman`).
- La deuxième colonne contient le prénom.
- La troisième colonne contient la taille (en centimètres).
- La quatrième colonne contient l'âge (en années).

## Filtres

### Sélection par expression régulière

Une expression régulière est une combinaison de caractères et de métacaractères (caractères ayant une signification spéciale) utilisée pour filtrer une chaîne de caractères cible. En anglais, une expression régulière se dit *regular expression* ou *regex*.

Avec `awk`, une utilisation des expressions régulières est de la forme : `awk '/regex/ {actions}' fichier`

On rappelle que les `{actions}` que nous verrons par la suite sont optionnelles. On peut donc écrire directement `awk '/regex/' fichier`

On effectue un test avec l'expression régulière sur chaque ligne et on affiche la ligne si le test est vérifié. La notion de colonne n'a pas encore d'importance ici. Par exemple, on cherche dans le jeu de données le mot clef `woman` et on affiche les lignes qui le contiennent :

In [ ]:
awk '/woman/' people.dat

Remarquez l'expression régulière `woman` qui se met entre `/ /`.

Par exemple, pour afficher les lignes qui contiennent le caractère `i` puis n'importe quel caractère deux fois (`..`) puis le caractère `e` :

In [ ]:
awk '/i..e/' people.dat

Remarque : dans une expression régulière, le métacaractère `.` correspond à n'importe quel caractère.

Voici un autre exemple qui affiche les lignes qui contiennent le caractère `s` suivi du caractère `i` ou `e` (les caractères `i` et `e` sont alors entre crochets) :

In [ ]:
awk '/s[ie]/' people.dat

### Sélection sur une colonne précise

Bien sur, si l'expression régulière n'est pas très précise, on affiche beaucoup de lignes :

In [ ]:
awk '/an/' people.dat

Dans l'exemple ci-dessus, l'expression régulière `an` est présente dans la 1re et la 2e colonne.

On peut alors demander à `awk` de vérifier l'expression régulière sur une colonne en particulier (ici la 2e) :

In [ ]:
awk '$2~/an/' people.dat

`awk` numérote automatique les colonnes de `$1` la première colonne à `$n` la n-ième colonne. 

La notation `$0` désigne toutes les colonnes (donc la ligne entière).

Voici comment sont numérotées les colonnes dans notre fichier `people.dat` :

```
      man     simon       175     33
      woman   clara       167     45
      man     serge       181     44
      woman   claire      174     31
      man     patrick     172     52
      woman   julie       168     37
      man     paul        185     29
      woman   jeanne      172     56
      man     baptiste    178     39
      woman   mathilde    168     46
      man     bob         186     33
$0 -- woman   elise       159     63
      |       |           |       |
      $1      $2          $3      $4
```

Exemple. Afficher les lignes pour lesquelles la seconde colonne débute par la lettre `p`. Le métacaractère `^` est utilisé pour indiquer le début de la colonne.

In [ ]:
awk '$2~/^p/' people.dat

Exemple. Afficher les lignes pour lesquelles la quatrième colonne est supérieure (strictement) à 50 :

In [ ]:
awk '$4 > 50' people.dat

Pour revenir à `$0`, les deux commandes suivantes sont équivalentes :

In [ ]:
awk '/is/' people.dat

et

In [ ]:
awk '$0~/is/' people.dat

### Opérateurs de comparaisons

Voici quelques opérateurs de comparaison pour les expressions régulières, les chaînes de caractères et les valeurs numériques.

Pour les expressions régulières ou les chaînes de caractères :

| opérateur | signification     |
|-----------|-------------------|
| `~`       | correspond        |
| `!~`      | ne correspond pas |

Pour les valeurs numériques :

| opérateur | signification        |
|-----------|----------------------|
| `==`      | égal à               |
| `!=`      | différent            |
| `>`       | supérieur à          |
| `>=`      | supérieur ou égale à |
| `<`       | inférieur à          |
| `<=`      | inférieur ou égale à |

Combinaison de plusieurs comparaisons :

| opérateur | signification |
|-----------|---------------|
| &&        | et            |
| &#921;&#921;     | ou            |

Exemple. Afficher les lignes pour lesquelles la première colonne débute par `m` (expression régulière `^m`) et la deuxième colonne contient `mo`.

In [ ]:
awk '$1~/^m/ && $2~/mo/' people.dat

Exemple. Afficher les lignes pour lesquelles la première colonne débute par `m` (expression régulière `^m`) et la troisième colonne est supérieure à 180. Pour la troisième colonne, on ne compare pas d'expression régulière entre `/ /` mais directement la valeur numérique.

In [ ]:
awk '$1~/^m/ && $3 > 180' people.dat

Exemple. Afficher les lignes pour lesquelles la première colonne débute par `m` (expression régulière `^m`) et la quatrième colonne est égale à 33.


In [ ]:
awk '$1~/^m/ && $4 == 33' people.dat

Exemple. Afficher les lignes pour lesquelles la quatrième colonne est inférieure (strictement) à 30 **ou** supérieure (strictement) à 60 :

In [ ]:
awk '$4 < 30 || $4 > 60' people.dat

## Actions

Pour mémoire, la syntaxe générale de `awk` est : **`awk [options] 'filtres {actions}' fichier`**

L'action la plus courante avec `awk` est l'affichage avec la commande `print`.

Exemple. Afficher la deuxième colonne des lignes qui contiennent le mot `woman` :

In [ ]:
awk '/woman/ {print $2}' people.dat

Même chose, précédée de la chaîne de caractères `prenom :` :

In [ ]:
awk '/woman/ {print "prenom :", $2}' people.dat

## Variables prédéfinies

`awk` fournit automatiquement un certain nombre de variables prédéfinies.


### Nombre de champs (colonnes) : `NF`

In [ ]:
awk '/paul/ {print NF}' people.dat

### Numéro de ligne : `NR`

La première ligne du fichier porte le numéro 1.

In [ ]:
awk 'NR>3 && NR<=5 {print $2}' people.dat

Exemple. Afficher les lignes paires :

In [ ]:
awk 'NR%2==0 {print $2}' people.dat

Remarque : l'opérateur modulo `%` renvoie le reste de la division entière. Ainsi `4%2` renvoie 0, `5%2` renvoie 1, `6%2` renvoie 0, `7%2` renvoie 1, etc.

### Variables crées à la volée

In [ ]:
awk '/woman/ {a=a+1; print a, $2}' people.dat

Dans cet exemple, la variable `a` n'existe pas a priori. Lorsque `awk` veut l'utiliser dans l'expression `a=a+1`, il initialise la variable `a` à la valeur `0`. Il l'utilise ensuite dans l'expression `print a, $2`.

Dans `awk`, deux actions sont séparées par le caractère `;`.

L'expression `a=a+1` est équivalente à `a++`, qui présente l'avantage d'être plus compacte :

In [ ]:
awk '/woman/ {a++; print a, $2}' people.dat

## BEGIN et END

Les mot-clefs `BEGIN` et `END` ont une signification très particulière puisque les actions qui les suivent ne sont exécutées qu'au début (`BEGIN`) ou à la fin (`END`) du parcours du fichier.

Exemple. Afficher la 2e colonne lorsque les lignes contiennent `woman`, puis le nombre total de lignes affichées.

In [ ]:
awk '/woman/ {a++; print $2} END {print "total:", a }' people.dat

L'action `{print "total:", a }` n'est exécutée que lorsque toutes les lignes du fichier `people.dat` ont été parcourues.

Autre exemple :

- afficher `women found:` avant la lecture du fichier,
- afficher la deuxième colonne des lignes du fichier `people.dat` qui contiennent `woman`,
- afficher `total: ` avec le nombre total de lignes sélectionnées après la lecture du fichier.

In [ ]:
awk 'BEGIN {print "women found:"} \
/woman/ {a++; print $2} \
END {print "total:", a }' people.dat

Remarque : le symbole `\` en fin de ligne permet d'écrire une commande sur plusieurs lignes.

Voilà encore deux derniers exemples pour terminer.

Calculer et afficher l'âge moyen des femmes :

In [ ]:
awk '/woman/ {age=age+$4; count++} \
END {print "mean age:", age/count}' people.dat

Calculer et afficher l'âge moyen des femmes et afficher le nombre de femmes :

In [ ]:
awk '/woman/ {age=age+$4; count++} \
END {print "total:", count; \
print "mean age:", age/count}' people.dat

# Script

Lorsque les instructions `awk` deviennent trop nombreuses, il est plus pratique d'écrire un script dédié.

Exemple. Calculer et afficher l'âge moyen des femmes et afficher le nombre de femmes.

Les instructions `awk` sont dans le fichier `mean_age_women.awk`  :

```
BEGIN {
age=0
count=0
}

/woman/ {
count++
age=age+$4
}

END {
print "total woman:", count
print "mean age:", age/count
}

```

Les blocs d'instructions sont ainsi plus lisibles. Le retour à la ligne suffit à séparer plusieurs actions (pas besoin de `;`).

Téléchargez le script `mean_age_women.awk` :

In [ ]:
wget https://raw.githubusercontent.com/pierrepo/unix/master/mean_age_women.awk

Ce script s'utilise en appelant `awk` avec l'option `-f` :

In [ ]:
awk -f mean_age_women.awk people.dat

# Séparateur de champ : option -F

Par défaut, `awk` suppose que les différentes colonnes (les différents champs) du fichier d'entrée sont séparées par des espaces ou des tabulations (une ou plusieurs.)

L'option `-F` (à ne pas confondre avec `-f`) définit le caractère qui sépare les différentes colonnes entre elles.

Dans notre exemple `people.dat`, les différentes colonnes sont séparées par des espaces (séparateur par défaut de `awk`).

Pour le vérifier, on peut demander à `awk` d'afficher le nombre de colonnes trouvées pour la première ligne uniquement :

In [ ]:
awk 'NR==1 {print NF}' people.dat

Si on demande à `awk` de lire le même fichier mais en prenant le caractère `,` comme séparateur, `awk` ne trouve plus qu'un seul champ (car il n'y a pas de `,` dans le fichier `people.dat`) :

In [ ]:
awk -F "," 'NR==1 {print NF}' people.dat

Dans un fichier au format [*tabulation-separated values*](https://fr.wikipedia.org/wiki/Tabulation-separated_values) (`.tsv`), les différentes colonnes sont séparées par le caractère tabulation.

Téléchargez par exemple le fichier `people.tsv` :

In [ ]:
wget https://raw.githubusercontent.com/pierrepo/unix/master/people.tsv

Puis affichez le contenu de ce fichier :

In [ ]:
cat people.tsv

Le caractère tabulation est un caractère « élastique », c'est-à-dire qu'il est affiché avec une taille variable correspondante à 1 ou plusieurs caractères. Regardez par exemple la première ligne du fichier `people.tsv` où les champs *man*, *simon*, *175* et *33* sont séparés les uns des autres par un seul caractère tabulation mais qui apparaît comme 1 ou plusieurs espaces.

Pour lire un tel fichier avec `awk`, l'option `-F` n'est pas nécessaire car la tabulation est aussi le séparateur par défaut reconnu par `awk` (comme l'espace).

In [ ]:
awk 'NR==1 {print NF}' people.tsv

Par contre, dans un fichier au format [*comma-separated values*](https://fr.wikipedia.org/wiki/Comma-separated_values) (`.csv`), les différents champs sont séparés par une virgule.

Téléchargez par exemple le fichier `people.csv` :

In [ ]:
wget https://raw.githubusercontent.com/pierrepo/unix/master/people.csv

Puis affichez le contenu de ce fichier :

In [ ]:
cat people.csv

Ici, l'utilisation de l'option `-F` est indispensable pour que `awk` parcourt correctement le fichier.

Voici ce que cela donnerait sans mentionner le séparateur de champs :

In [ ]:
awk 'NR==1 {print NF}' people.csv

Et quand on indique explictement le séparateur de champs avec `-F` :

In [ ]:
awk -F "," 'NR==1 {print NF}' people.csv

Pour terminer, voici l'effet de l'utilisation de `-F` sur un cas concret :

In [ ]:
awk '/woman/ {print "prenom :", $2}' people.csv

In [ ]:
awk -F "," '/woman/ {print "prenom :", $2}' people.csv

En conclusion, `awk` est un outil très puissant pour manipuler des fichiers tabulés, c'est-à-dire avec une structure en lignes et en colonnes (même si une telle structure n'est pas toujours évidente).

Soyez par contre toujours très attentif au format des fichiers que vous manipulez.
